In [79]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns

In [80]:
test = pd.read_csv('test_mriguel.csv')
train = pd.read_csv('train_mriguel.csv')

In [81]:
test.isnull().sum().any()>0

False

In [82]:
y=train['SalePrice']
X=train.drop('SalePrice',axis=1)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=32)

Defining a method to evalutate our models : 

In [83]:
def evaluate(model, test_features, test_labels):
    
    from sklearn.metrics import mean_squared_log_error
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    rmsle = np.sqrt(mean_squared_log_error(test_labels , predictions ))
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    print('rmsle = {:0.4f}%.'.format(rmsle))
    
    return accuracy

In [84]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


# RandomForestRegressor GridSearchCV

In [86]:
rfr=RandomForestRegressor()

param_grid = { 
    'n_estimators': [10,15,80,100,200,300],
    'max_depth': [10, 100, 150,300,500],
}

grid_search = GridSearchCV(estimator=rfr, param_grid=param_grid, cv=5, n_jobs=-1,verbose=1)
gs_fit = grid_search.fit(X_train, y_train)
best_grid = grid_search.best_estimator_


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   22.1s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.7min finished


In [87]:
evaluate(best_grid, X_test, y_test)


Model Performance
Average Error: 16935.6410 degrees.
Accuracy = 89.43%.
rmsle = 0.1541%.


In [106]:
pred=best_grid.predict(test)

In [108]:
pred=pd.DataFrame(pred)

In [110]:
test.head()

,Unnamed: 0,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,...,SaleCondition,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,1458,896,0,0,1,2,0,3,1,468.0,...,4,8,120,1,5,882.0,140,1961,1961,4
1,1459,1329,0,0,1,3,0,3,1,923.0,...,4,8,0,1,6,1329.0,393,1958,1958,4
2,1460,928,701,0,1,3,0,3,1,791.0,...,4,8,0,1,6,928.0,212,1997,1998,4
3,1461,926,678,0,1,3,0,3,1,602.0,...,4,8,0,1,7,926.0,360,1998,1998,4
4,1462,1280,0,0,1,2,4,3,1,263.0,...,4,8,144,1,5,1280.0,0,1992,1992,4


In [111]:
result=pd.DataFrame()
result['Id']=test['Unnamed: 0']
result['SalePrice']=pred
result.index=range(1461,result.shape[0]+1461)
result.head()

,Id,SalePrice
1461,1458,126041.036868
1462,1459,153196.215977
1463,1460,172457.834416
1464,1461,181555.760847
1465,1462,196258.060215


In [117]:
result.head()

,Id,SalePrice
1461,1461,126041.036868
1462,1462,153196.215977
1463,1463,172457.834416
1464,1464,181555.760847
1465,1465,196258.060215


In [116]:
result['Id']=range(1461,result.shape[0]+1461)

In [118]:
result.to_csv('submit5.csv',index=False)